## Упражнение 03: Агрегации

In [23]:
import pandas as pd
import sqlite3

## создаем подключение к базе данных с помощью библиотеки sqlite3

In [24]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## получить схему таблицы test

In [25]:
pd.read_sql('pragma table_info(test)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## получить только первые 10 строк таблицы test, чтобы проверить, как выглядит таблица

In [26]:
pd.read_sql('select * from test limit 10;', conn)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## найти среди всех пользователей минимальное значение дельты между первым коммитом пользователя и крайним сроком соответствующей лабораторной работы, используя только один запрос

- сделайте это, присоединившись к таблице с крайними сроками
- разница должна отображаться в часах
- не принимайте во внимание лабораторную работу «проект 1», у нее более длительные сроки выполнения, и она будет выделяться
- значение должно быть сохранено в фрейме данных df_min с соответствующим uid

In [27]:
query = """
        select uid, min(delta) as min_deadline from (
        select uid, first_commit_ts, datetime(deadlines.deadlines, 'unixepoch'),
               (deadlines - unixepoch(first_commit_ts)) / 3600 as delta
        from test
        join deadlines on deadlines.labs = test.labname
        where labname != 'project1')
        """

df_min = pd.read_sql(query, conn)
df_min

,uid,min_deadline
0,user_25,2


## сделайте то же самое, но для максимума, используя только один запрос, имя dataframe будет df_max

In [28]:
query = """
        select uid, max(delta) as max_deadline from (
        select uid, first_commit_ts, datetime(deadlines.deadlines, 'unixepoch'),
               (deadlines - unixepoch(first_commit_ts)) / 3600 as delta
        from test
        join deadlines on deadlines.labs = test.labname
        where labname != 'project1')
        """

df_max = pd.read_sql(query, conn)
df_max

,uid,max_deadline
0,user_30,202


## сделайте то же самое, но для среднего значения, используя только один запрос, на этот раз ваш фрейм данных не должен включать столбец uid, а имя фрейма данных — df_avg

In [29]:
query = """
        select avg(delta) as avg_deadline from (
        select uid, first_commit_ts, datetime(deadlines.deadlines, 'unixepoch'),
               (deadlines - unixepoch(first_commit_ts)) / 3600 as delta
        from test
        join deadlines on deadlines.labs = test.labname
        where labname != 'project1')
        """

df_avg = pd.read_sql(query, conn)
df_avg

,avg_deadline
0,89.125


## мы хотим проверить гипотезу о том, что пользователи, которые посетили новостную ленту всего несколько раз, имеют более низкую дельту между первым коммитом и дедлайном. Для этого нужно рассчитать коэффициент корреляции между количеством просмотров страниц и разницей

- Используя только один запрос, создайте таблицу со столбцами: uid, avg_diff, pageviews
uid — это uid, которые существуют в тесте
- avg_diff — это средняя разница между первым коммитом и крайним сроком выполнения лабораторных работ на пользователя.
- просмотры страниц — это количество посещений новостной ленты на одного пользователя
- не принимайте во внимание лабораторную работу «проект1»
- сохраните его в dataframe views_diff
- используйте метод Pandas corr() для расчета коэффициента корреляции между количеством просмотров страниц и разницей

In [30]:
query = """
        select uid, round(avg(delta), 2) as avg_diff, max(page_views) as pageviews from (
        select uid, first_commit_ts, datetime(deadlines.deadlines, 'unixepoch'),
               (deadlines - unixepoch(first_commit_ts)) / 3600 as delta,
               (select count(*) from pageviews
                where test.uid = pageviews.uid) as page_views
        from test
        join deadlines on deadlines.labs = test.labname
        where labname != 'project1')
        group by uid
        """

views_diff = pd.read_sql(query, conn)
views_diff.set_index('uid', inplace=True)

print(views_diff)
views_diff.corr()

         avg_diff  pageviews
uid                         
user_1      64.40         28
user_10     74.80         89
user_14    159.00        143
user_17     61.60         47
user_18      5.67          3
user_19     98.75         16
user_21     95.50         10
user_25     92.60        179
user_28     86.40        149
user_3     105.40        317
user_30    145.25          3


,avg_diff,pageviews
avg_diff,1.000000,0.279734
pageviews,0.279734,1.000000


## закрыть соединение

In [31]:
conn.close()